# Utiliser Ray-Serve pour "Servir" le modèle Camembert :

Ce Notebook explore l'utilisation de Ray-Serve pour pouvoir créer une API et ainsi pouvoir appeler le modèle Camembert depuis n'importe où. Notamment depuis le site web INSPIRE. 

In [2]:
#Importing Librairies
from transformers import CamembertForSequenceClassification,CamembertTokenizer, Trainer
import ray
from ray import serve
import requests
import argparse
import torch
import numpy as np

Initilization d'arguments :

In [3]:
args=argparse.Namespace()
use_gpu = torch.cuda.is_available()
#Use this line if you want ot use a GPU (if available)
# args.device = torch.device("cuda" if use_gpu else "cpu")
#Use this one to use the CPU
args.device = torch.device("cpu")

Démarrez le Client Ray Serve :

In [5]:
client=serve.start()

2021-03-08 16:13:48,850	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=10247) 2021-03-08 16:13:50,841	INFO http_state.py:67 -- Starting HTTP proxy with name 'dSsfpl:SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-node:192.168.1.25-0' on node 'node:192.168.1.25-0' listening on '127.0.0.1:8000'
(pid=10249) INFO:     Started server process [10249]


On définit ici une classe que l'on va utiliser dans Ray serve. 

On charge dans l'initilialisation le modèle choisi.

La fonction call permet d'utiliser le modèle sur un input et de retourner la réponse du modèle.

In [6]:
class predict_class:
    def __init__(self,args):
        self.args=args
        self.model=CamembertForSequenceClassification.from_pretrained("/home/amaury/Documents/project_a1/camembert-v1")
        self.tokenizer=CamembertTokenizer.from_pretrained("camembert-base")

        trainer=Trainer(
            model=self.model
        )
        self.trainer=trainer
        self.model.to(args.device)

    def __call__(self,request):
        input=await request.body()
        text=input.decode("utf-8")
        
        tokenized=self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")

        result=self.model(**tokenized)
        class_input=np.argmax(result.logits.data.numpy())
        return({"class":str(class_input)})

In [7]:
#Run those lines in case of changes in the class to be able to create a new backend and endpoint.
client.delete_endpoint("classpredict")
client.delete_backend("classpredict")

On crée maintenant l'API proprement dit :

In [8]:
# client.create_backend("classpredict", predict_class, args, ray_actor_options={"num_gpus": 1})
client.create_backend("classpredict", predict_class, args)
client.create_endpoint("classpredict",backend="classpredict", route="/classpredict",methods=["GET","POST"])

(pid=10247) 2021-03-08 16:14:14,059	INFO controller.py:178 -- Registering route '/classpredict' to endpoint 'classpredict' with methods '['GET', 'POST']'.


Que l'on peut appeler ici : 

In [9]:
payload="Bonjour".encode("utf-8")
r=requests.post("http://127.0.0.1:8000/classpredict",data=payload)
r.content

(pid=10249) 2021-03-08 16:14:29,975	INFO router.py:248 -- Endpoint classpredict doesn't exist, waiting for registration.


b'{\n  "class": "0"\n}'